In [1]:
import json
import pandas as pd
from kafka import KafkaConsumer

In [3]:
pip install azure-storage-file-datalake

Note: you may need to restart the kernel to use updated packages.


In [3]:
consumer = KafkaConsumer(
    'demo-topic',
    bootstrap_servers='localhost:9092',
    auto_offset_reset='earliest',
    enable_auto_commit=True,
    group_id='my-consumer-group'
)


In [7]:
from azure.storage.filedatalake import DataLakeServiceClient

# Azure ADLS Gen2 credentials
storage_account_name = "financestoragedatalake"
storage_account_key = "+PXKMWq2qGxY1lhlFYXTTobpsqMGMtkka9e9yV2XisGsX7U5qrzsbkpSDJyOAk9MUsN3retAaAHq+AStKQJw/A=="

# Authenticate Data Lake client
service_client = DataLakeServiceClient(
    account_url=f"https://financestoragedatalake.dfs.core.windows.net",
    credential=storage_account_key
)

file_system_client = service_client.get_file_system_client(file_system="bronze")
directory_client = file_system_client.get_directory_client("kafka-output")

file_client = directory_client.create_file("kafka_data.jsonl")  # writing as line-delimited JSON

In [ ]:
messages = []

for msg in consumer:
    decoded_message = msg.value.decode('utf-8')  # bytes ➔ str
    messages.append(decoded_message)  # already JSON string

if messages:
    data = "\n".join(messages)

    # Upload to ADLS Gen2
    file_client.append_data(data=data, offset=0, length=len(data))
    file_client.flush_data(len(data))

    print(f"✅ Written {len(messages)} messages to ADLS Gen2")
else:
    print("⚠ No messages found in topic.")


In [ ]:
from kafka import KafkaConsumer

consumer = KafkaConsumer(
    'demo-topic',
    bootstrap_servers='localhost:9092',
    auto_offset_reset='earliest',
    enable_auto_commit=True
)

# Read messages
for message in consumer:
    print(message.value.decode('utf-8'))
    # Optional exit condition to break

# 🔴 **Close the consumer properly**
consumer.close()

print("✅ Kafka Consumer closed.")


In [ ]:
consumer.close()